# TensorFlow Fundamentals

In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we are going to cover:
* Introducion to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself!

## Introduction to Tensors

In [2]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2024-10-11 09:08:47.378511: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 09:08:52.155438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 09:08:53.890011: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 09:08:54.349964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 09:08:57.827476: I tensorflow/core/platform/cpu_feature_guar

2.17.0


### Creating tensors with tf.constant()

In [3]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

I0000 00:00:1728608994.977588    2049 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728608999.831057    2049 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728608999.831117    2049 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728608999.846399    2049 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728608999.846478    2049 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [5]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [6]:
# Check the dimensions of our vector
vector.ndim

1

In [7]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                                  [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [8]:
# Check the dimensions of our matrix
matrix.ndim

2

In [9]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                                                [8., 2.],
                                                [8., 9.]],
                                                dtype=tf.float16) # specify the data type with dtype parameter
another_matrix

2024-10-11 09:10:00.404065: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 8.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [10]:
# What's the dimensions of another_matrix?
another_matrix.ndim

2

In [11]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                                   [4, 5, 6]],
                                  [[7, 8, 9],
                                   [10, 11, 12]],
                                  [[13, 14, 15],
                                   [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [12]:
# Dimensions of our tensor
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with tf.Variable()

In [13]:
# Create teh same tensor with tf.Variable()
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [14]:
# Let's try change one of the elements in our changeable tensor

# TypeError: 'ResourceVariable' object does not support item assignment
# changeable_tensor[0] = 7

In [15]:
# How about we try .assign()
changeable_tensor[0].assign(7)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [16]:
# Let's try change our unchangeable tensor

# TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment
# unchangeable_tensor[0] = 7

# AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'
# unchangeable_tensor[0].assign(7)

🔑 Note: Rarely in practice will you need to decide whether to use tf.constant or tf.Variable to create tensors, as TensorFlow does this for you. However, if in doubt, use tf.constant and change it later if needed.

### Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers.

In [17]:
# Create 2 random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [18]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect the learning)
not_shuffled = tf.constant([[10, 7],
                                            [3, 4],
                                            [2, 5]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [19]:
# Shuffle our non-shuffled tensor (1st dimension)
tf.random.shuffle(not_shuffled) # result should be different in each run

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [20]:
# If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### Other ways to make tensors

In [21]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [22]:
# Create a tensor of all zeros
tf.zeros(shape=[3, 4])

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn Numpy arrays into tensors

The main difference between Numpy arrays and TensorFlow tensors is that tensors can be run on a GPU computing.

In [23]:
import numpy as np

np_A = np.arange(1, 25, dtype=np.int32) # create a Numpy array between 1 and 25
np_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) #non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [24]:
tf_A = tf.constant(np_A)
tf_A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [25]:
tf_B = tf.constant(np_A, shape=(2, 3, 4))
tf_B

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

In [26]:
tf_B.ndim

3

## Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [27]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros([2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [28]:
# Get various attributes of our tensor
print(f'Shape of rank_4_tensor: {rank_4_tensor.shape}')
print(f"Datatype of every element in rank_4_tensor: {rank_4_tensor.dtype}")
print(f'Dimentions (rank) of rank_4_tensor: {rank_4_tensor.ndim}')
print(f'Elements along the 0 axis of rank_4_tensor: {rank_4_tensor.shape[0]}')
print(f'Elements along the last axis of rank_4_tensor: {rank_4_tensor.shape[-1]}')
print(f'Size (total number of element) of rank_4_tensor: {tf.size(rank_4_tensor)}')

Shape of rank_4_tensor: (2, 3, 4, 5)
Datatype of every element in rank_4_tensor: <dtype: 'float32'>
Dimentions (rank) of rank_4_tensor: 4
Elements along the 0 axis of rank_4_tensor: 2
Elements along the last axis of rank_4_tensor: 5
Size (total number of element) of rank_4_tensor: 120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [29]:
# Get the first 2 elemnts of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [30]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [31]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7], [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [32]:
# Get the last item of each of row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [33]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [34]:
# Alternative to add in extra dimension
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [35]:
# Alternative to add in extra dimension
tf.expand_dims(rank_2_tensor, axis=0) # expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

## Manipulating tensors (tensor operations)

### Basic operations

`+`, `-`, `*`, `/`

In [36]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [37]:
# Multiplication
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [38]:
# Substraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [39]:
# We can use the tensorflow built-in function too
# tf math function might be faster on GPU
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [40]:
tf.add(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [41]:
tf.divide(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

In [42]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

### Matrix multiplication

In machine learning, matrix multiplication is one of the most common tensor operations.

There are 2 rules our tensors (matrices) need to fulfill if we are going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the outside dimensions

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will tranpose (rather than reshape) one of the tensors to get satisify the matrix multiplication rules.

In [43]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [44]:
# Matrix multiplication in TensorFlow (dot product)
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [45]:
# Matrix multiplication with Python operator "*" (element-wise multiplication)
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [46]:
# Matrix multiplication with Python operator "@" (dot product)
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [47]:
# Create a tensor (3, 2) tensor
X = tf.constant([[1, 2], [3, 4], [5, 6]])

# Create another (3, 2) tensor
Y = tf.constant([[7, 8], [9, 10], [11, 12]])

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [48]:
# Matrix multiplication of X and Y (note that the shape of X and Y is (3, 2), (3, 2) so we need to transpose one of them)
# Changing the shape of Y to (2, 3)
print(X.shape, tf.reshape(Y, shape=(2, 3)).shape)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

(3, 2) (2, 3)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [49]:
# Try to change the shape of X instead of Y
# note the result is different, the resulting matrix has the shape of the outer dimensions of the matrices being multiplied
print(tf.reshape(X, shape=(2, 3)).shape, Y.shape)
tf.reshape(X, shape=(2, 3)) @ Y

(2, 3) (3, 2)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

#### Reshape vs. Transpose

**Reshape:** Changes the dimensions of a tensor.

**Transpose:** Flips the axes of a tensor (swap dimensions).

In [50]:
# Can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [51]:
# Try matrix multiplication with transpose rather than reshape
print(tf.transpose(X).shape, Y.shape)
tf.matmul(tf.transpose(X), Y)

(2, 3) (3, 2)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

#### The dot product

Matrix multiplication is also referred as the dot product.

You can perform matrix multiplication using:

* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [52]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [53]:
# Perform matrix multiplication between X and Y (transpose Y)
print(X.shape, tf.transpose(Y).shape)
tf.tensordot(X, tf.transpose(Y), axes=1)

(3, 2) (2, 3)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [54]:
# Perform matrix multiplication between X and Y (reshaped Y)
print(X.shape, tf.reshape(Y, shape=(2, 3)).shape)
tf.tensordot(X, tf.reshape(Y, shape=(2, 3)), axes=1)

(3, 2) (2, 3)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [55]:
# Check the values of Y, reshape Y and transposed Y
print("Normal Y:")
print(Y, "\n") # "\n" is for newline

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, (2, 3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


### Changing datatype of a tensor

Sometimes you'll want to alter the default datatype of your tensor.

This is common when you want to compute using less precision (e.g. 16-bit floating point numbers vs. 32-bit floating point numbers).

Computing with less precision is useful on devices with less computing capacity such as mobile devices (because the less bits, the less space the computations require).

You can change the datatype of a tensor using `tf.cast()`.

In [56]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])

# Create a new tensor with default datatype (int32)
C = tf.constant([1, 7])
B, C

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 7], dtype=int32)>)

In [57]:
# Change from float32 to float16 (reduced precision)
B = tf.cast(B, dtype=tf.float16)
B

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>

In [58]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 7.], dtype=float32)>

In [59]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1., 7.], dtype=float16)>

### Tensor aggregation

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [60]:
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [61]:
# Get the absolute values
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:

* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [62]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([59, 14, 27, 60, 49, 43, 13, 23, 53, 62, 24, 18, 74, 79, 47, 95, 95,
       49, 44, 81, 15, 39, 52, 16, 62, 65,  1, 27, 56, 49, 50, 76,  7, 63,
       30, 27, 32, 26, 44, 36, 29, 51, 25, 64, 13, 72, 31, 19, 49, 11])>

In [63]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [64]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [65]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=95>

In [66]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [67]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2146>

**Exercise:** With what we have just learnd, find the variance and standard deviation of our `E` tensor using TensorFlow methods.

In [68]:
# Find the variance
float32_E = tf.cast(E, dtype=tf.float32)
tf.math.reduce_variance(float32_E)

<tf.Tensor: shape=(), dtype=float32, numpy=530.6336>

In [69]:
# Find the standard deviation
tf.math.reduce_std(float32_E)

<tf.Tensor: shape=(), dtype=float32, numpy=23.035486>

### Finding the positional minimum and maximum of a tensor (argmin and argmax)

In [70]:
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [71]:
# Find the minimum value's index and the minimum value
print(f"Index of the minimum value: {tf.argmin(F).numpy()}")
print(f"Minimum value: {tf.reduce_min(F).numpy()}")

Index of the minimum value: 16
Minimum value: 0.009463667869567871


In [72]:
# Find the largest value's index and the largest value
print(f'The maximum value of F is at position: {tf.argmax(F)}')
print(f'The maximum value of F is: {tf.reduce_max(F)}')

The maximum value of F is at position: 42
The maximum value of F is: 0.967138409614563


### Squeezing a tensor (removing all 1-dimension axes)

In [73]:
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.68789124, 0.48447883, 0.9309944 , 0.252187  , 0.73115396,
           0.89256823, 0.94674826, 0.7493341 , 0.34925628, 0.54718256,
           0.26160395, 0.69734323, 0.11962581, 0.53484344, 0.7148968 ,
           0.87501776, 0.33967495, 0.17377627, 0.4418521 , 0.9008261 ,
           0.13803864, 0.12217975, 0.5754491 , 0.9417181 , 0.9186585 ,
           0.59708476, 0.6109482 , 0.82086265, 0.83269787, 0.8915849 ,
           0.01377225, 0.49807465, 0.57503664, 0.6856195 , 0.75972784,
           0.908944  , 0.40900218, 0.8765154 , 0.53890026, 0.42733097,
           0.401173  , 0.66623247, 0.16348064, 0.18220246, 0.97040176,
           0.06139731, 0.53034747, 0.9869994 , 0.4746945 , 0.8646754 ]]]]],
      dtype=float32)>

In [74]:
G_squeezed = tf.squeeze(G)
G_squeezed

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.68789124, 0.48447883, 0.9309944 , 0.252187  , 0.73115396,
       0.89256823, 0.94674826, 0.7493341 , 0.34925628, 0.54718256,
       0.26160395, 0.69734323, 0.11962581, 0.53484344, 0.7148968 ,
       0.87501776, 0.33967495, 0.17377627, 0.4418521 , 0.9008261 ,
       0.13803864, 0.12217975, 0.5754491 , 0.9417181 , 0.9186585 ,
       0.59708476, 0.6109482 , 0.82086265, 0.83269787, 0.8915849 ,
       0.01377225, 0.49807465, 0.57503664, 0.6856195 , 0.75972784,
       0.908944  , 0.40900218, 0.8765154 , 0.53890026, 0.42733097,
       0.401173  , 0.66623247, 0.16348064, 0.18220246, 0.97040176,
       0.06139731, 0.53034747, 0.9869994 , 0.4746945 , 0.8646754 ],
      dtype=float32)>

### One-hot encoding

One-hot encoding converts categorical data into a binary matrix where each category is represented by a unique column with a value of 1 for the presence of the category and 0 otherwise.

In [75]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [76]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=len(some_list), on_value="on", off_value="off")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'on', b'off', b'off', b'off'],
       [b'off', b'on', b'off', b'off'],
       [b'off', b'off', b'on', b'off'],
       [b'off', b'off', b'off', b'on']], dtype=object)>

### Squaring, log, square root


* `tf.square()` - get the square of every value in a tensor.
* `tf.sqrt()` - get the squareroot of every value in a tensor (note: the elements need to be floats or this will error).
* `tf.math.log()` - get the natural log of every value in a tensor (elements need to floats).


In [77]:
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [78]:
# Square
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [79]:
# Square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [80]:
# Logarithm
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Manipulating tf.Variable tensors

Tensors created with tf.Variable() can be changed in place using methods such as:

* `.assign()` - assign a different value to a particular index of a variable tensor.
* `.add_assign()` - add to an existing value and reassign it at a particular index of a variable tensor.


In [81]:
# Create a variable tensor
I = tf.Variable(np.arange(0, 5))
I

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([0, 1, 2, 3, 4])>

In [82]:
# Assign the final value a new value of 50
I.assign([0, 1, 2, 3, 50])

<tf.Variable 'UnreadVariable' shape=(5,) dtype=int64, numpy=array([ 0,  1,  2,  3, 50])>

In [83]:
# The change happens in place (the last value is now 50, not 4)
I

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([ 0,  1,  2,  3, 50])>

In [84]:
# Add 10 to every element in I
I.assign_add([10, 10, 10, 10, 10])
# Again, the change happens in place
I

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([10, 11, 12, 13, 60])>

## Tensors and NumPy

We've seen some examples of tensors interact with NumPy arrays, such as, using NumPy arrays to create tensors.

Tensors can also be converted to NumPy arrays using:

* np.array() - pass a tensor to convert to an ndarray (NumPy's main datatype).
* tensor.numpy() - call on a tensor to convert to an ndarray.

Doing this is helpful as it makes tensors iterable as well as allows us to use any of NumPy's methods on them.

In [85]:
# Create a tensor directly from a Numpy array
J = tf.constant(np.array([3., 7., 10.]))
J, type(J)

(<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>,
 tensorflow.python.framework.ops.EagerTensor)

In [86]:
# Convert out tensor back to a Numpy array
np_J = J.numpy()
np_J, type(np_J)

(array([ 3.,  7., 10.]), numpy.ndarray)

In [87]:
# 2nd way to convert tensor to Numpy array
np_J_2 = np.array(J)
np_J_2, type(np_J_2)

(array([ 3.,  7., 10.]), numpy.ndarray)

In [88]:
# The default data types of each are slightly different
np_J = tf.constant(np.array([3., 7., 10.]))
ts_J = tf.constant([3., 7., 10.])

np_J.dtype, ts_J.dtype

(tf.float64, tf.float32)

## Finding access to GPUs

You can check if you've got access to a GPU using tf.config.list_physical_devices().

In [89]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [90]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [91]:
!nvidia-smi

Fri Oct 11 09:10:06 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:65:00.0 Off |                  N/A |
| N/A   54C    P3             12W /   70W |    6303MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 🛠 TensorFlow Fundamentals Exercises

1. Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().
2. Find the shape, rank and size of the tensors you created in 1.
3. Create two tensors containing random values between 0 and 1 with shape [5, 300].
4. Multiply the two tensors you created in 3 using matrix multiplication.
5. Multiply the two tensors you created in 3 using dot product.
6. Create a tensor with random values between 0 and 1 with shape [224, 224, 3].
7. Find the min and max values of the tensor you created in 6 along the first axis.
8. Create a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
9. Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
10. One-hot encode the tensor you created in 9.

In [92]:
# 1. Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().
vector = tf.constant([1, 2, 3])
scalar = tf.constant(7)
matrix = tf.constant([[1, 2, 3], [4, 5, 6]])
tensor = tf.constant([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]], [[13, 14, 15], [16, 17, 18]]])

vector, scalar, matrix, tensor

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=7>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
 array([[[ 1,  2,  3],
         [ 4,  5,  6]],
 
        [[ 7,  8,  9],
         [10, 11, 12]],
 
        [[13, 14, 15],
         [16, 17, 18]]], dtype=int32)>)

In [93]:
# 2. Find the shape, rank and size of the tensors you created in 1.
tensor.shape, tensor.ndim, tf.size(tensor)

(TensorShape([3, 2, 3]), 3, <tf.Tensor: shape=(), dtype=int32, numpy=18>)

In [94]:
# 3. Create two tensors containing random values between 0 and 1 with shape [5, 300].
tf.random.set_seed(42)
tensor_1 = tf.random.uniform(shape=[5, 300])
tensor_2 = tf.random.uniform(shape=[5, 300])

tensor_1, tensor_2

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.6645621 , 0.44100678, 0.3528825 , ..., 0.31410468, 0.7593535 ,
         0.03699052],
        [0.532024  , 0.29129946, 0.10571766, ..., 0.54052293, 0.31425726,
         0.2200619 ],
        [0.08404207, 0.03614604, 0.97732127, ..., 0.21516645, 0.9786098 ,
         0.00726748],
        [0.7396945 , 0.6653172 , 0.0787828 , ..., 0.7117733 , 0.07013571,
         0.9409125 ],
        [0.15861344, 0.12024033, 0.27218235, ..., 0.8824879 , 0.1432488 ,
         0.44135118]], dtype=float32)>,
 <tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.68789124, 0.48447883, 0.9309944 , ..., 0.6920762 , 0.33180213,
         0.9212563 ],
        [0.27369928, 0.10631859, 0.6218617 , ..., 0.4382149 , 0.30427706,
         0.51477313],
        [0.00920248, 0.37280262, 0.8177401 , ..., 0.56786287, 0.49201214,
         0.9892651 ],
        [0.88608265, 0.08672249, 0.12160683, ..., 0.91770685, 0.72545695,
         0.8280058 ],
        [0.36690

In [97]:
# 4. Multiply the two tensors you created in 3 using matrix multiplication.
tf.matmul(tensor_1, tf.reshape(tensor_2, shape=(300, 5)))

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[76.27554 , 79.17827 , 77.82468 , 79.390816, 79.37842 ],
       [72.70598 , 73.674446, 73.71501 , 74.18447 , 73.72763 ],
       [75.73994 , 76.49056 , 74.43401 , 78.80334 , 77.701225],
       [68.98679 , 75.53526 , 74.58016 , 76.19388 , 75.24947 ],
       [75.563354, 81.79909 , 78.68235 , 80.158264, 80.48132 ]],
      dtype=float32)>

In [96]:
# 5. Multiply the two tensors you created in 3 using dot product.
tf.tensordot(tensor_1, tf.transpose(tensor_2), axes=1)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[80.33309 , 73.40606 , 77.15882 , 73.98497 , 80.90113 ],
       [75.14486 , 68.80302 , 74.241905, 71.84128 , 75.60254 ],
       [79.75885 , 75.64509 , 77.79609 , 74.74909 , 80.56139 ],
       [75.084404, 69.06293 , 74.30793 , 72.27636 , 76.05752 ],
       [85.05516 , 74.266624, 78.00625 , 74.8864  , 83.135666]],
      dtype=float32)>

In [98]:
# 6. Create a tensor with random values between 0 and 1 with shape [224, 224, 3].
tensor_3 = tf.random.uniform(shape=[224, 224, 3])
tensor_3

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.7413678 , 0.62854624, 0.01738465],
        [0.3431449 , 0.51063764, 0.3777541 ],
        [0.07321596, 0.02137029, 0.2871771 ],
        ...,
        [0.98953485, 0.45382905, 0.2006687 ],
        [0.6295223 , 0.4937899 , 0.01816809],
        [0.95386636, 0.11542463, 0.85691285]],

       [[0.78435016, 0.7826872 , 0.87936425],
        [0.24906898, 0.3207239 , 0.10955775],
        [0.543224  , 0.7151396 , 0.40334642],
        ...,
        [0.2445668 , 0.01746976, 0.9036933 ],
        [0.02975535, 0.592268  , 0.9877522 ],
        [0.36701274, 0.33112562, 0.5638567 ]],

       [[0.15829337, 0.7288823 , 0.3366307 ],
        [0.70792687, 0.16910625, 0.9429966 ],
        [0.10120225, 0.5919596 , 0.8687303 ],
        ...,
        [0.28134012, 0.10011208, 0.37038183],
        [0.77874243, 0.05421627, 0.4664607 ],
        [0.2549187 , 0.7968637 , 0.83405185]],

       ...,

       [[0.32922816, 0.06343532, 0.23936498],
        [0.42

In [100]:
# 7. Find the min values of the tensor you created in 6 along the first axis.
tf.reduce_min(tensor_3, axis=0)

<tf.Tensor: shape=(224, 3), dtype=float32, numpy=
array([[2.67446041e-03, 2.14838982e-03, 1.05965137e-03],
       [5.47564030e-03, 2.21121311e-03, 4.15408611e-03],
       [3.72385979e-03, 5.20217419e-03, 9.02771950e-04],
       [1.11603737e-03, 6.35766983e-03, 3.18384171e-03],
       [2.10273266e-03, 5.84840775e-04, 1.05011463e-03],
       [4.26018238e-03, 1.58730745e-02, 1.21641159e-03],
       [6.51884079e-03, 1.17063522e-03, 2.71463394e-03],
       [5.67364693e-03, 8.54074955e-03, 1.52909756e-02],
       [7.19797611e-03, 8.43882561e-04, 1.91795826e-03],
       [4.71234322e-04, 6.39832020e-03, 2.17664242e-03],
       [2.41947174e-03, 5.75447083e-03, 1.54391527e-02],
       [9.96232033e-04, 1.06847286e-03, 1.08550787e-02],
       [7.00187683e-03, 3.46624851e-03, 2.44665146e-03],
       [1.11377239e-03, 7.05718994e-05, 2.76803970e-03],
       [3.22699547e-03, 9.34267044e-03, 4.85146046e-03],
       [2.55906582e-03, 3.21245193e-03, 2.09597349e-02],
       [4.42552567e-03, 6.46495819e-03

In [101]:
# 7. Find the min values of the tensor you created in 6 along the first axis.
tf.reduce_max(tensor_3, axis=0)

<tf.Tensor: shape=(224, 3), dtype=float32, numpy=
array([[0.99845374, 0.99930906, 0.99262166],
       [0.99466574, 0.99434555, 0.9957906 ],
       [0.9993377 , 0.9988344 , 0.99788725],
       [0.9930023 , 0.99388456, 0.9973532 ],
       [0.99890304, 0.99934196, 0.999063  ],
       [0.99909663, 0.9912498 , 0.9989666 ],
       [0.9984517 , 0.9993125 , 0.9972266 ],
       [0.9851161 , 0.9997518 , 0.9938358 ],
       [0.99558496, 0.9875896 , 0.99800336],
       [0.99865353, 0.9981276 , 0.99659836],
       [0.99160516, 0.99429524, 0.99986553],
       [0.99120665, 0.9974369 , 0.9837067 ],
       [0.9998193 , 0.9986278 , 0.9997519 ],
       [0.9990151 , 0.9997431 , 0.99959373],
       [0.99833953, 0.99908006, 0.9974154 ],
       [0.9997325 , 0.9948616 , 0.99942064],
       [0.9960431 , 0.99533176, 0.99374545],
       [0.99750674, 0.9957348 , 0.99729204],
       [0.9953059 , 0.99919796, 0.99888754],
       [0.99667954, 0.9962187 , 0.99953544],
       [0.9983853 , 0.9915079 , 0.99962735],
     

In [103]:
# 8. Create a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
tensor_4 = tf.random.uniform(shape=[1, 224, 224, 3])
tensor_4

<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[8.0315602e-01, 4.9777734e-01, 3.7054038e-01],
         [9.1186738e-01, 6.3764203e-01, 1.8209696e-01],
         [6.3791955e-01, 2.7701473e-01, 4.2271137e-02],
         ...,
         [1.0830712e-01, 4.5979273e-01, 2.5716281e-01],
         [8.7138689e-01, 1.8434000e-01, 4.4757760e-01],
         [7.4110627e-02, 9.0852141e-01, 5.3693414e-01]],

        [[5.5596435e-01, 6.8776274e-01, 7.6051474e-02],
         [1.6737962e-01, 7.1785092e-01, 2.7642274e-01],
         [2.6995218e-01, 3.2203627e-01, 8.8224900e-01],
         ...,
         [4.8168826e-01, 5.0150025e-01, 8.6756039e-01],
         [4.1261053e-01, 1.2770486e-01, 5.8186901e-01],
         [2.5495613e-01, 3.9036548e-01, 9.8529553e-01]],

        [[8.0935180e-01, 1.9740558e-01, 3.5899937e-01],
         [1.1216915e-01, 9.1016293e-04, 3.6382091e-01],
         [5.1202202e-01, 3.9188230e-01, 8.8335538e-01],
         ...,
         [2.0133841e-01, 9.1663551e-01, 1.9890130e-01],


In [104]:
tensor_4_squeezed = tf.squeeze(tensor_4)
tensor_4_squeezed

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[8.0315602e-01, 4.9777734e-01, 3.7054038e-01],
        [9.1186738e-01, 6.3764203e-01, 1.8209696e-01],
        [6.3791955e-01, 2.7701473e-01, 4.2271137e-02],
        ...,
        [1.0830712e-01, 4.5979273e-01, 2.5716281e-01],
        [8.7138689e-01, 1.8434000e-01, 4.4757760e-01],
        [7.4110627e-02, 9.0852141e-01, 5.3693414e-01]],

       [[5.5596435e-01, 6.8776274e-01, 7.6051474e-02],
        [1.6737962e-01, 7.1785092e-01, 2.7642274e-01],
        [2.6995218e-01, 3.2203627e-01, 8.8224900e-01],
        ...,
        [4.8168826e-01, 5.0150025e-01, 8.6756039e-01],
        [4.1261053e-01, 1.2770486e-01, 5.8186901e-01],
        [2.5495613e-01, 3.9036548e-01, 9.8529553e-01]],

       [[8.0935180e-01, 1.9740558e-01, 3.5899937e-01],
        [1.1216915e-01, 9.1016293e-04, 3.6382091e-01],
        [5.1202202e-01, 3.9188230e-01, 8.8335538e-01],
        ...,
        [2.0133841e-01, 9.1663551e-01, 1.9890130e-01],
        [8.0388057e-01

In [106]:
# 9. Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
tensor_5 = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
tf.argmax(tensor_5)

<tf.Tensor: shape=(), dtype=int64, numpy=9>

In [107]:
# 10. One-hot encode the tensor you created in 9.
tf.one_hot(tensor_5, depth=10)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>